# 05 • Reading Analysis
OLS (clustered) and MixedLM; BH-FDR.

In [ ]:
from pathlib import Path
import pandas as pd
from pcs_toolbox.analysis import fit_ols_clustered, fit_mixedlm
proc = Path('data/processed')
df = pd.read_csv(proc/'zuco_aligned.csv')
df = df.dropna(subset=['TRT'])
# Simple OLS clustered by Subject
m_ols = fit_ols_clustered(df, 'TRT ~ GD + FFD', cluster='Subject')
print(m_ols.summary().tables[1])


In [ ]:
# MixedLM with Subject random intercept; SentenceID as variance component if available
try:
    m_mixed = fit_mixedlm(df, 'TRT ~ GD + FFD', group='Subject', vc='SentenceID')
    print(m_mixed.summary())
except Exception as e:
    print('MixedLM failed:', e)


In [ ]:
# Save a simple bar plot of OLS coefficients as a figure
import matplotlib.pyplot as plt, os
coef = m_ols.params if 'm_ols' in globals() else None
if coef is not None:
    fig, ax = plt.subplots(figsize=(5,3))
    coef.drop(labels=[c for c in coef.index if c.startswith('Intercept')], errors='ignore').plot(kind='bar', ax=ax)
    ax.set_title('OLS coefficients: TRT ~ GD + FFD')
    ax.set_ylabel('Estimate')
    os.makedirs('figures', exist_ok=True)
    fig.tight_layout()
    fig.savefig('figures/F2_reading_coeffs.png', dpi=150)
    plt.close(fig)
